In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("all_processed.csv")
df.head(2)

,Unnamed: 0,Unnamed: 0.1,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,best compani work,peopl smart friendli,bureaucraci slow thing,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,move speed light burn inevit,food food food cafe main campu mtv alon mini k...,work life balanc balanc perk benefit illus kee...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...


In [3]:
df = df[df["summary"].notna()]
df = df[df["pros"].notna()]
df = df[df["cons"].notna()]

In [4]:
df["text"] = df["summary"].str.cat(df["pros"].str.cat(df["cons"],sep=" "),sep=" ")

In [5]:
df[["summary","pros","cons","text"]].head()

,summary,pros,cons,text
0,best compani work,peopl smart friendli,bureaucraci slow thing,best compani work peopl smart friendli bureauc...
1,move speed light burn inevit,food food food cafe main campu mtv alon mini k...,work life balanc balanc perk benefit illus kee...,move speed light burn inevit food food food ca...
2,great balanc big compani secur fun fast move p...,softwar engin among king hill googl engin driv...,becom larger come grow pain bureaucraci slow r...,great balanc big compani secur fun fast move p...
3,best place work also demand,find well regard compani actual deserv hype ge...,live sf commut take hour hour way shuttl somet...,best place work also demand find well regard c...
4,uniqu one kind dream job,googl world everi compani lot peopl seriou gap...,work mtv hq given work howev passion want adva...,uniqu one kind dream job googl world everi com...


In [18]:
np.mean(df["text"].str.len())

198.47529723770077

In [6]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(df["overall-ratings"])

In [7]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#cv = CountVectorizer()
cv = TfidfVectorizer()

In [8]:
X = cv.fit_transform(df["text"].tolist())
X.shape

(67118, 23341)

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=4)

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

pipe_knn = make_pipeline(StandardScaler(with_mean=False), KNeighborsClassifier(n_neighbors=5))

In [11]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe_knn, X, y, cv=kf)

array([0.25893921, 0.27234803, 0.24418951, 0.24389154, 0.25089392,
       0.2443385 , 0.23688915, 0.2409118 , 0.24959023, 0.24109671])

In [12]:
from sklearn.linear_model import LogisticRegression

pipe_lr = make_pipeline(StandardScaler(with_mean=False), LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))
cross_val_score(pipe_lr, X, df["overall-ratings"], cv=kf)

/home/ras/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ras/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ras/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ras/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/ras/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", 

array([0.41895113, 0.41716329, 0.41507747, 0.41030989, 0.4272944 ,
       0.41388558, 0.4056913 , 0.40867104, 0.4114141 , 0.41543734])

In [22]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
#Xs = pca.fit_transform(X)

In [23]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

pca = TruncatedSVD(10)
Xs = pca.fit_transform(X)

In [15]:
cross_val_score(pipe_knn, Xs, y, cv=kf)

array([0.25461859, 0.26400477, 0.25506555, 0.26623957, 0.25923719,
       0.2568534 , 0.2602801 , 0.26653754, 0.26046789, 0.25063329])

In [16]:
pca.transform(X[2])

array([[ 0.32737182, -0.04704281, -0.08027708, -0.13544934,  0.02325402,
         0.04087802,  0.0408246 ,  0.09603416, -0.05851127,  0.00301722]])

In [20]:
from sklearn.linear_model import LinearRegression

pipe_lr = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
cross_val_score(pipe_lr, Xs, df["overall-ratings"], cv=kf)

array([0.17670599, 0.17453031, 0.17190291, 0.18439016, 0.1761775 ,
       0.17486661, 0.17250953, 0.16827775, 0.18185334, 0.14941941])

In [21]:
pipe_lr = make_pipeline(StandardScaler(with_mean=False), LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))
cross_val_score(pipe_lr, Xs, df["overall-ratings"], cv=kf)

array([0.41746126, 0.44025626, 0.43161502, 0.42908224, 0.43087008,
       0.43399881, 0.42327175, 0.43146603, 0.42810311, 0.41275518])

In [24]:
from sklearn.tree import DecisionTreeClassifier

pipe_lr = make_pipeline(StandardScaler(with_mean=False), DecisionTreeClassifier())
cross_val_score(pipe_lr, Xs, df["overall-ratings"], cv=kf)

array([0.34982122, 0.35861144, 0.35443981, 0.3568236 , 0.34252086,
       0.35101311, 0.36188915, 0.35146007, 0.35091641, 0.34450902])

In [25]:
pipe_lr = make_pipeline(StandardScaler(with_mean=False), DecisionTreeClassifier())
cross_val_score(pipe_lr, X, df["overall-ratings"], cv=kf)

array([0.37604291, 0.38125745, 0.38960072, 0.37544696, 0.38885578,
       0.3806615 , 0.38319428, 0.3931764 , 0.37907912, 0.37788705])